In [1]:
%load_ext autoreload
%autoreload 2

from gnn_tracking.metrics.losses import CondensationLossTiger, CondensationLossRG
from gnn_tracking.training.callbacks import PrintValidationMetrics
from gnn_tracking.training.tc import TCModule
from gnn_tracking.models.graph_construction import MLGraphConstructionFromChkpt
from gnn_tracking.models.track_condensation_networks import PreTrainedECGraphTCN
import torch

In [2]:
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [3]:

model = PreTrainedECGraphTCN(
      ec=None,
      node_indim= 38,
      edge_indim= 76,
      h_dim= 192,
      e_dim= 192,
      hidden_dim= 192,
      h_outdim= 24,
      L_hc= 5,
      alpha_latent= 0.5,
      n_embedding_coords= 24,
)
preproc = MLGraphConstructionFromChkpt(
    ml_chkpt_path="/home/kl5675/Documents/23/git_sync/hyperparameter_optimization2/scripts/full_detector/lightning_logs/merciful-reindeer-of-coffee/checkpoints/epoch=79-step=72000.compat.ckpt",
    max_num_neighbors= 15,
    max_radius= 1.,
    use_embedding_features= True,
    build_edge_features= True,
)


/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:198: Attribute 'hc_in' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['hc_in'])`.
[13:17:18] DEBUG: Getting class MLModule from module gnn_tracking.training.ml
[13:17:27] DEBUG: Loading checkpoint /home/kl5675/Documents/23/git_sync/hyperparameter_optimization2/scripts/full_detector/lightning_logs/merciful-reindeer-of-coffee/checkpoints/epoch=79-step=72000.compat.ckpt
[13:17:27] DEBUG: Getting class GraphConstructionFCNN from module gnn_tracking.models.graph_construction
[13:17:27] DEBUG: Getting class GraphConstructionHingeEmbeddingLoss from module gnn_tracking.metrics.losses
[13:17:27] DEBUG: Getting class GraphConstructionKNNScanner from module gnn_tracking.graph_construction.k_scanner
[13:17:27] DEBUG: Checkpoint loaded. Model ready to go.


In [4]:

condensation_loss = CondensationLossTiger(
      q_min= 0.01,
      max_n_rep=100_000,
)


In [4]:
condensation_loss = CondensationLossRG(
      max_num_neighbors=50,
)

In [5]:
oc = TCModule(
    model=model,
    preproc=preproc,
    lw_repulsive=0.74,
    loss_fct=condensation_loss,
    cluster_scanner=None,
)

In [6]:
from gnn_tracking.utils.loading import TrackingDataModule

dm = TrackingDataModule(
    identifier="point_clouds_v10",
    train=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_1/"
        ],
        sample_size=1000,
        # If you run into memory issues, reduce this
    ),
    val=dict(
        dirs=[
            "/scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_9/"
        ],
        stop=5
    ),
)


In [7]:
import pytorch_lightning as pl

trainer = pl.Trainer(
    max_epochs=1,
    callbacks=[PrintValidationMetrics()],
    # fast_dev_run=True,
)


/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:
trainer.fit(oc, dm)



You are using a CUDA device ('NVIDIA A100 80GB PCIe') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
[13:17:37] INFO: DataLoader will load 900 graphs (out of 900 available).
[13:17:37] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_1/data21000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_1/data21999_s0.pt
[13:17:37] INFO: DataLoader will load 5 graphs (out of 1000 available).
[13:17:37] DEBUG: First graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_9/data29000_s0.pt, last graph is /scratch/gpfs/IOJALVO/gnn-tracking/object_condensation/point_clouds_v10/part_9/data29004_s0.pt
LOCAL_RANK: 0 - C

Sanity Checking: |                                                                                            …

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=2` in the `DataLoader` to improve performance.
/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=2` in the `DataLoader` to improve performance.


Training: |                                                                                                   …

/scratch/gpfs/kl5675/micromamba/envs/gnn/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
